In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from urllib.request import urlopen
import json
from scipy.interpolate import CubicSpline

In [2]:
with open("../.fmp_api.key", "r") as f:
    api_key = f.read().strip()

url = f"https://financialmodelingprep.com/stable/treasury-rates?apikey={api_key}"
response = urlopen(url)
par_rates = pd.DataFrame(json.loads(response.read().decode("utf-8")))

In [3]:
par_rates

,date,month1,month2,month3,month6,year1,year2,year3,year5,year7,year10,year20,year30
0,2025-07-01,4.32,4.42,4.40,4.29,3.98,3.78,3.75,3.84,4.03,4.26,4.79,4.78
1,2025-06-30,4.28,4.45,4.41,4.29,3.96,3.72,3.68,3.79,3.98,4.24,4.79,4.78
2,2025-06-27,4.19,4.49,4.39,4.26,3.97,3.73,3.72,3.83,4.03,4.29,4.85,4.85
3,2025-06-26,4.11,4.49,4.39,4.26,3.96,3.70,3.68,3.79,4.00,4.26,4.81,4.81
4,2025-06-25,4.21,4.43,4.38,4.26,3.99,3.74,3.74,3.83,4.05,4.29,4.83,4.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2025-04-07,4.36,4.36,4.29,4.14,3.86,3.73,3.72,3.82,3.97,4.15,4.61,4.58
59,2025-04-04,4.36,4.36,4.28,4.14,3.86,3.68,3.66,3.72,3.84,4.01,4.44,4.41
60,2025-04-03,4.36,4.38,4.31,4.20,3.92,3.71,3.68,3.75,3.88,4.06,4.51,4.49
61,2025-04-02,4.38,4.34,4.32,4.24,4.04,3.91,3.89,3.95,4.07,4.20,4.58,4.54


In [4]:
tenors = [(t, float(t.replace("month", "").replace("year","")) / \
           (12 if "month" in t else 1)) for t in par_rates.columns[1:]]

xt = np.concatenate((np.array([t[1] for t in tenors[:4]]), np.arange(1, 30.5, 0.5)))

tenors, xt

([('month1', 0.08333333333333333),
  ('month2', 0.16666666666666666),
  ('month3', 0.25),
  ('month6', 0.5),
  ('year1', 1.0),
  ('year2', 2.0),
  ('year3', 3.0),
  ('year5', 5.0),
  ('year7', 7.0),
  ('year10', 10.0),
  ('year20', 20.0),
  ('year30', 30.0)],
 array([ 0.08333333,  0.16666667,  0.25      ,  0.5       ,  1.        ,
         1.5       ,  2.        ,  2.5       ,  3.        ,  3.5       ,
         4.        ,  4.5       ,  5.        ,  5.5       ,  6.        ,
         6.5       ,  7.        ,  7.5       ,  8.        ,  8.5       ,
         9.        ,  9.5       , 10.        , 10.5       , 11.        ,
        11.5       , 12.        , 12.5       , 13.        , 13.5       ,
        14.        , 14.5       , 15.        , 15.5       , 16.        ,
        16.5       , 17.        , 17.5       , 18.        , 18.5       ,
        19.        , 19.5       , 20.        , 20.5       , 21.        ,
        21.5       , 22.        , 22.5       , 23.        , 23.5       ,
        24

In [5]:
def interpolate_curve(tenors, rates, xt):
    spline = CubicSpline(
        [t[1] for t in tenors], rates, bc_type="natural"
    )
    yt = spline(xt)
    return (pd.DataFrame(
            {
                "xt": xt,
                "par_rate": yt,
            }
        ).assign(key_rate = lambda df_: [1 if df_.xt.iloc[i] in [t[1] for t in tenors] else 0 for i in range(df_.shape[0])])
    )

interpolated_curve = interpolate_curve(tenors, par_rates.iloc[0, 1:], xt)

In [6]:
interpolated_curve

,xt,par_rate,key_rate
0,0.083333,4.320000,1
1,0.166667,4.420000,1
2,0.250000,4.400000,1
3,0.500000,4.290000,1
4,1.000000,3.980000,1
...,...,...,...
58,28.000000,4.807418,0
59,28.500000,4.800911,0
60,29.000000,4.794106,0
61,29.500000,4.787103,0


In [7]:
def bootstrapSpotRates(par_rates):
    """
    Compute the spot rate for a given maturity i using the par rates.
    """
    spot_rates = []
    for i in range(par_rates.shape[0]):
        if par_rates.xt.iloc[i] < 1:
            spot_rates.append(par_rates.par_rate.iloc[i])
        else:
            fv = 100
            c = par_rates.par_rate.iloc[i] / 2.
            xt = par_rates.xt.iloc[:i+1]; xt = xt[xt >= 0.5]
            rt = spot_rates[3:]
            df = np.sum([1. / (1 + rt[j] / 100.) ** xt.iloc[j] for j in range(len(rt))])
            spot_rates.append((((fv + c) / (fv - c * df)) ** (1. / xt.iloc[-1]) - 1) * 100)
    return pd.Series(spot_rates, index=par_rates.index)
    
interpolated_curve["spot_rate"] = bootstrapSpotRates(interpolated_curve)

In [8]:
interpolated_curve

,xt,par_rate,key_rate,spot_rate
0,0.083333,4.320000,1,4.320000
1,0.166667,4.420000,1,4.420000
2,0.250000,4.400000,1,4.400000
3,0.500000,4.290000,1,4.290000
4,1.000000,3.980000,1,4.016916
...,...,...,...,...
58,28.000000,4.807418,0,5.022598
59,28.500000,4.800911,0,5.005064
60,29.000000,4.794106,0,4.986939
61,29.500000,4.787103,0,4.968435


In [12]:
with open("../var/jun2025/spot_rates.json", "w") as f:
    f.write(interpolated_curve.loc[:, ["xt", "spot_rate"]].to_json(orient="records", indent=2))

In [39]:
curve_10y = interpolated_curve.iloc[3:23].loc[:, ["xt", "par_rate", "spot_rate"]].copy()

def pv_bond(coupon_rate, spot_rates, face_value=100):
    """
    Compute the present value of a bond given its coupon rate, maturity, and face value.
    """
    coupon = coupon_rate / 200 * face_value
    cash_flows = np.array([coupon] * (spot_rates.size -1) + [face_value + coupon])
    discount_factors = (1 + curve_10y.spot_rate / 100) ** -curve_10y.xt
    pv = np.sum(cash_flows * discount_factors)
    return pv

pv_bond(4.5, curve_10y.spot_rate, 1000)

np.float64(1014.6440366040135)

In [46]:
np.arange(0.5, 10.5, 0.5).size

20

In [51]:
def pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value=100):
    """
    Compute the present value of a bond given its coupon rate, yield to maturity, and maturity.
    """
    coupon = coupon_rate / 200 * face_value
    cash_flows = np.array([coupon] * (n_maturity * 2 - 1) + [face_value + coupon])
    discount_factors = (1 + ytm / 100.) ** -np.arange(0.5, n_maturity + 0.5, 0.5)
    pv = np.sum(cash_flows * discount_factors)
    return pv

def pv_bond_prime(coupon_rate, ytm, n_maturity, face_value=100):
    """
    Compute the derivative of the present value of a bond with respect to its coupon rate.
    """
    coupon = coupon_rate / 200 * face_value
    cash_flows = np.array([coupon * i for i in np.arange(0.5, n_maturity, 0.5)] + [n_maturity * (face_value + coupon)])
    discount_factors = (1 + ytm / 100) ** -(np.arange(0.5, n_maturity + 0.5, 0.5) + 1)
    pv = np.sum(cash_flows * discount_factors)
    
    return pv

def yield_to_maturity(bond_price, coupon_rate, n_maturity, face_value=100, tolerance=1e-6, max_iterations=1000):
    """
    Use the Newton-Raphson method to find the coupon rate that makes the present value of a bond equal to a target value.
    """
    ytm = coupon_rate
    ytm_old = 0
    for i in range(max_iterations):
        if abs(ytm - ytm_old) < tolerance:
            return ytm
        ytm_old = ytm
        pv = pv_bond_ytm(coupon_rate, ytm, n_maturity, face_value)
        ytm = ytm_old - (bond_price - pv) / pv_bond_prime(coupon_rate, ytm, n_maturity, face_value)
    raise ValueError("Newton-Raphson method did not converge")

target_pv = 985
ytm = yield_to_maturity(target_pv, 5., 10, 1000)
ytm


np.float64(5.261562284024409)

In [52]:
pv_bond_ytm(5., 5.2615, 10, 100)

np.float64(98.50119787017108)